In [1]:
#Carregando libs
library(readxl)
library(caret)
library(MASS)
library(glmnet)
source('utils.R',local=TRUE)

Loading required package: lattice
Loading required package: ggplot2
Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-13

Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess

Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following object is masked from ‘package:glmnet’:

    auc

The following objects are masked from ‘package:stats’:

    cov, smooth, var



In [2]:
#LEITURA E TRATAMENTO DO DATASET

#Leitura da base de dados
base6 <- read_excel("base6.xls",sheet=2)

#Removendo id de cliente
base6 = base6[,-1]

#Transformando censura como variavel categoria
base6$censura = as.factor(base6$censura)

#Transformando o índice B2B como variavel categorica
base6$indB2B = as.factor(base6$indB2B)

#Aplicando padronização nas variaveis numericas
base6[,c(1,3,5,6,7,8)] = scale(base6[,c(1,3,5,6,7,8)])

In [3]:
#SELEÇÃO DE VARIÁVEIS
#OLHAR EXPLICAÇÃO NO RELATÓRIO

#Removendo Duracao pois foi visto que causa problemas
#principalmente na regressão logistica(separação perfeita) e no qda
base6 = base6[,-1]

#Regressão logistica
logistic_regression = glm(formula = censura ~ .,data=base6, family = binomial(link = logit))

#Iremos criar modelos com os métodos de seleção de variáveis na regressão logistica
#para averigurar quais delas parecem ser mais importantes
model1 = step(logistic_regression,k = log(nrow(base6)))

model2 = stepAIC(logistic_regression,direction = c('backward'))

model3 = stepAIC(logistic_regression,direction = c('forward'))

model4 = stepAIC(logistic_regression,direction = c('both'))

x = model.matrix(censura ~ ., data = base6)[, -1]
y <- ifelse(base6$censura == 0, 0, 1)

#Regularização com método Lasso
model5 <- cv.glmnet(x, y, family = 'binomial', alpha = 1, nfolds = 10)

#Coeficientes de cada modelo
coef(model1)
coef(model2)
coef(model3)
coef(model4)
coef(model5)

#Removendo o indice B2B porque aparenta nao afetar o modelo
base6 = base6[,-3]

#Removendo TotalFreq porque aparenta nao afetar o modelo
base6 = base6[,-6]

#Removendo receita porque aparenta nao afetar o modelo
base6 = base6[,-3]

Start:  AIC=421.86
censura ~ valorGasto + indB2B + receita + nEmpregados + TotalProdutos + 
    TotalFreq

                Df Deviance    AIC
- indB2B         1   379.78 417.07
- TotalFreq      1   384.13 421.41
<none>               378.36 421.86
- receita        1   389.68 426.97
- TotalProdutos  1   475.89 513.17
- nEmpregados    1   512.97 550.26
- valorGasto     1   513.76 551.05

Step:  AIC=417.07
censura ~ valorGasto + receita + nEmpregados + TotalProdutos + 
    TotalFreq

                Df Deviance    AIC
- TotalFreq      1   385.35 416.42
<none>               379.78 417.07
- receita        1   392.12 423.19
- TotalProdutos  1   478.34 509.41
- valorGasto     1   516.14 547.22
- nEmpregados    1   517.64 548.71

Step:  AIC=416.42
censura ~ valorGasto + receita + nEmpregados + TotalProdutos

                Df Deviance    AIC
<none>               385.35 416.42
- receita        1   396.58 421.44
- TotalProdutos  1   481.22 506.08
- valorGasto     1   520.69 545.54
- nEmpregados 

(Intercept)    valorGasto       receita   nEmpregados TotalProdutos 
    0.2357714    -1.6932539     0.4312884     1.6423281     1.2864626

(Intercept)    valorGasto       receita   nEmpregados TotalProdutos 
    0.2348601    -1.7173994     0.4566788     1.6732152     1.3232132 
    TotalFreq 
    0.3021141

(Intercept)    valorGasto       indB2B1       receita   nEmpregados 
    0.4363966    -1.7137970    -0.3201989     0.4405998     1.6583777 
TotalProdutos     TotalFreq 
    1.3188667     0.3091756

(Intercept)    valorGasto       receita   nEmpregados TotalProdutos 
    0.2348601    -1.7173994     0.4566788     1.6732152     1.3232132 
    TotalFreq 
    0.3021141

7 x 1 sparse Matrix of class "dgCMatrix"
                        1
(Intercept)    0.19690167
valorGasto    -1.09206205
indB2B1        .         
receita        0.15036703
nEmpregados    1.07110715
TotalProdutos  0.81438947
TotalFreq      0.04396586

In [4]:
#AVALIANDO OS MODELOS

#Vetores que conteram as medias
meanACC1 = meanACC2 = meanACC3 = meanACC4 = meanACC5 = c()
meanSEN1 = meanSEN2 = meanSEN3 = meanSEN4 = meanSEN5 = c()
meanESP1 = meanESP2 = meanESP3 = meanESP4 = meanESP5 = c()
meanAUC1 = meanAUC2 = meanAUC3 = meanAUC4 = meanAUC5 = c()

for(i in 1:50){
    #Indices para separar dataset
    index = createDataPartition(y=base6$censura,p=0.6,list=FALSE)

    #Separando dados em treino e teste
    train = base6[index,]
    test = base6[-index,]

    #MODELOS DE CLASSIFICACAO

    #Regressão logistica
    logistic_regression = glm(formula = censura ~ .,data=train, family = binomial(link = logit))

    #Knn
    knn <- train(censura ~ ., data = train, method = "knn", tuneLength = 10,trControl = trainControl(method = "cv"))    

    #Discriminante linear
    lda = train(censura ~ ., data=train, method="lda", trControl = trainControl(method = "cv"))

    #Discriminante quadratico
    qda = train(censura ~ ., data=train, method="qda", trControl = trainControl(method = "cv"))

    #Arvore de decisao
    dtree = train(censura ~ ., data=train, method="rpart", trControl = trainControl(method = "cv"))
    
    #AVALIAÇÃO DOS MODELOS

    #Matriz de confusao e métricas 
    confMat = confusion_matrix(logistic_regression,test,type='glm')
    metric1 = metric_confusion_matrix(confMat)

    confMat = confusion_matrix(knn,test,type='caret')
    metric2 = metric_confusion_matrix(confMat)

    confMat = confusion_matrix(lda,test,type='caret')
    metric3 = metric_confusion_matrix(confMat)

    confMat = confusion_matrix(qda,test,type='caret')
    metric4 = metric_confusion_matrix(confMat)

    confMat = confusion_matrix(dtree,test,type='caret')
    metric5 = metric_confusion_matrix(confMat)

    #Calculando AUC de cada modelo
    auc1 = extract_auc(logistic_regression,test,type='glm')
    auc2 = extract_auc(knn,test,type='caret')
    auc3 = extract_auc(lda,test,type='caret')
    auc4 = extract_auc(qda,test,type='caret')
    auc5 = extract_auc(dtree,test,type='caret')
    
    #Media da acuracia, sensibilidade e especificidade
    meanACC1 = c(meanACC1,metric1$accuracy)
    meanACC2 = c(meanACC2,metric2$accuracy)
    meanACC3 = c(meanACC3,metric3$accuracy)
    meanACC4 = c(meanACC4,metric4$accuracy)
    meanACC5 = c(meanACC5,metric5$accuracy)
    
    meanSEN1 = c(meanSEN1,metric1$sens)
    meanSEN2 = c(meanSEN2,metric2$sens)
    meanSEN3 = c(meanSEN3,metric3$sens)
    meanSEN4 = c(meanSEN4,metric4$sens)
    meanSEN5 = c(meanSEN5,metric5$sens)
    
    meanESP1 = c(meanESP1,metric1$espec)
    meanESP2 = c(meanESP2,metric2$espec)
    meanESP3 = c(meanESP3,metric3$espec)
    meanESP4 = c(meanESP4,metric4$espec)
    meanESP5 = c(meanESP5,metric5$espec)
    
    #Media dos AUC's
    meanAUC1 = c(meanAUC1,extract_auc(logistic_regression,test,type='glm'))
    meanAUC2 = c(meanAUC2,extract_auc(knn,test,type='caret'))
    meanAUC3 = c(meanAUC3,extract_auc(lda,test,type='caret'))
    meanAUC4 = c(meanAUC4,extract_auc(qda,test,type='caret'))
    meanAUC5 = c(meanAUC5,extract_auc(knn,test,type='caret'))
}

In [5]:
#Apanhado geral dos resultados
resume = round(data.frame(rbind(mean(meanACC1),mean(meanACC2),mean(meanACC3),mean(meanACC4),mean(meanACC5))),3)
resume = cbind(resume,round(data.frame(rbind(mean(meanSEN1),mean(meanSEN2),mean(meanSEN3),mean(meanSEN4),mean(meanSEN5))),3))
resume = cbind(resume,round(data.frame(rbind(mean(meanESP1),mean(meanESP2),mean(meanESP3),mean(meanESP4),mean(meanESP5))),3))
resume = cbind(resume,round(data.frame(rbind(mean(meanAUC1),mean(meanAUC2),mean(meanAUC3),mean(meanAUC4),mean(meanAUC5))),3))

names(resume) = c('Accuracy','Sens','Espec','AUC')
rownames(resume) = c('Logistic Regression','knn','Lda','Qda','Dtree')
resume

,Accuracy,Sens,Espec,AUC
Logistic Regression,0.808,0.831,0.782,89.260
knn,0.825,0.881,0.760,90.952
Lda,0.807,0.834,0.774,89.263
Qda,0.833,0.849,0.816,91.650
Dtree,0.775,0.808,0.736,90.952


In [6]:
#TREINAMENTO COM A BASE COMPLETA

#Regressão logistica
logistic_regression = glm(formula = censura ~ .,data=base6, family = binomial(link = logit))

#Knn
knn <- train(censura ~ ., data = base6, method = "knn", tuneLength = 10,trControl = trainControl(method = "cv"))    

#Discriminante linear
lda = train(censura ~ ., data=base6, method="lda", trControl = trainControl(method = "cv"))

#Discriminante quadratico
qda = train(censura ~ ., data=base6, method="qda", trControl = trainControl(method = "cv"))

#Arvore de decisao
dtree = train(censura ~ ., data=base6, method="rpart", trControl = trainControl(method = "cv"))

In [7]:
#CUSTOS

#Custo de um TN é 0.5 vezes maior que um TP
cost(knn,base6,type='caret',c(0.5, 268/500))
cost(qda,base6,type='caret',c(0.5, 268/500))


#Custo de um TN é 2 vezes maior que um TP
cost(knn,base6,type='caret',c(2, 268/500))
cost(qda,base6,type='caret',c(2, 268/500))

threshold specificity sensitivity 
      0.405      83.190      85.821

threshold specificity sensitivity 
      0.451      85.345      83.582

threshold specificity sensitivity 
      0.595      70.259      95.149

threshold specificity sensitivity 
      0.703      69.397      95.896